In [ ]:
# Dataset: Centro Federal de Educação Tecnológica de Minas Gerais - CEFET-MG
Fonte: Dados Abertos - Governo Federal
Plano de Dados Abertos 2022-2024 / Última atualização: 14/02/2023
Relação de todos os alunos ingressantes e formandos, por ciclo acadêmico.

Disponível em https://dados.gov.br/dados/conjuntos-dados/12-alunos

In [22]:
import pandas as pd
from datetime import datetime

# Converter o conteúdo para um DataFrame
df = pd.read_csv(r'../PDA_2022-2024_1.2_Alunos_Anonimo.csv', sep=';', encoding='latin1', skiprows=2)

# Exibir as primeiras linhas do DataFrame
display(df.head(10))

,Ingressante/Concluinte,Matrícula,Nome,Nível de Ensino,Curso,Campus,Situação de Matrícula,Período de Ingresso,Data da Apuração
0,Ingressante,NaN,ACHILLES M S,TÉCNICO INTEGRADO,TÉCNICO DE NÍVEL MÉDIO EM MECATRÔNICA,CAMPUS VARGINHA,ATIVO,2022-0,06/06/22
1,Ingressante,NaN,ADELINO P O,TÉCNICO INTEGRADO,TÉCNICO DE NÍVEL MÉDIO EM ELETROTÉCNICA,CAMPUS NEPOMUCENO,ATIVO,2022-0,06/06/22
2,Ingressante,NaN,ADLER L R,TÉCNICO INTEGRADO,TÉCNICO EM MEIO AMBIENTE,CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
3,Ingressante,NaN,ADRIANA D O,TÉCNICO INTEGRADO,TÉCNICO EM ELETRÔNICA (SUBSEQUENTE),CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
4,Ingressante,NaN,ADRIELLE A P S,TÉCNICO INTEGRADO,TÉCNICO EM HOSPEDAGEM,CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
5,Ingressante,NaN,ADRIELLE E D S,TÉCNICO INTEGRADO,TÉCNICO EM TRÂNSITO,CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
6,Ingressante,NaN,ADRYAN C V D F,TÉCNICO INTEGRADO,TÉCNICO EM MECATRÔNICA,CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
7,Ingressante,NaN,ÁGATA O R,TÉCNICO INTEGRADO,TÉCNICO DE NÍVEL MÉDIO EM PRODUÇÃO DE MODA - I...,CAMPUS DIVINÓPOLIS,ATIVO,2022-0,06/06/22
8,Ingressante,NaN,ÁGATHA I M,TÉCNICO INTEGRADO,TÉCNICO EM EQUIPAMENTOS BIOMÉDICOS,CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22
9,Ingressante,NaN,AIRTON F L,TÉCNICO INTEGRADO,TÉCNICO EM ELETRÔNICA (SUBSEQUENTE),CAMPUS NOVA SUÍÇA,ATIVO,2022-0,06/06/22


In [5]:
# Filtrar o DataFrame para o nível de ensino superior e situação de matrícula ativa ou concluinte.
df_superior_ativo = df[(df['Nível de Ensino'] == 'GRADUAÇÃO') & ((df['Situação de Matrícula'] == 'ATIVO') | (df['Ingressante/Concluinte'] == 'Concluinte'))] 


# Uma função de Data: calcular o tempo de conclusão do curso
def calcular_tempo_conclusao(data_ingresso, data_conclusao):
    
    # Função condicional: tratar valores com mês 0 como mês 1
    if data_ingresso.endswith('-0'):
        data_ingresso = data_ingresso[:-2] + '-1'
    if data_conclusao.endswith('-0'):
        data_conclusao = data_conclusao[:-2] + '-1'

    # Função de conversão: converter strings em objetos datetime
    data_ingresso = datetime.strptime(data_ingresso, '%Y-%m')
    data_conclusao = datetime.strptime(data_conclusao, '%Y-%m')
    if data_conclusao.month == 0:
        data_conclusao = data_conclusao.replace(month=1)
    return (data_conclusao.year - data_ingresso.year) * 12 + data_conclusao.month - data_ingresso.month

# Aplicar a função ao DataFrame: função condicional apenas para concluintes
df_superior_ativo['Tempo de Estimado de Conclusão (anos)'] = df_superior_ativo.apply(
    lambda row: calcular_tempo_conclusao(row['Situação de Matrícula'], row['Período de Ingresso']) / 12 if row['Ingressante/Concluinte'] == 'Concluinte' else None, axis=1
)


C:\Users\roche\AppData\Local\Temp\ipykernel_25276\4152303023.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_superior_ativo['Tempo de Estimado de Conclusão (anos)'] = df_superior_ativo.apply(


In [6]:

# Duas funções de Agregação
df_grupo = df_superior_ativo.groupby(["Ingressante/Concluinte", 'Curso']).agg({
	'Tempo de Estimado de Conclusão (anos)': lambda x: round(x.mean(), 1),
	'Nome': 'count'
}).reset_index()

# Renomear as colunas agregadas
df_grupo.rename(columns={'Tempo de Estimado de Conclusão (anos)': 'Tempo Médio de Conclusão (anos)', 'Nome': 'Contagem de Alunos'}, inplace=True)
df_grupo.sort_values(by='Contagem de Alunos', ascending=False, inplace=True)

# Exibir as primeiras linhas do DataFrame filtrado
display(df_grupo.head(10))

,Ingressante/Concluinte,Curso,Tempo Médio de Conclusão (anos),Contagem de Alunos
12,Concluinte,ENGENHARIA INDUSTRIAL MECÂNICA,5.7,1322
11,Concluinte,ENGENHARIA INDUSTRIAL ELÉTRICA,5.5,1256
8,Concluinte,ENGENHARIA DE PRODUÇÃO CIVIL,5.5,914
17,Concluinte,PROGRAMA ESPECIAL DE FORMAÇÃO PEDAGÓGICA DE DO...,0.0,554
10,Concluinte,ENGENHARIA ELÉTRICA,5.8,537
14,Concluinte,ENGENHARIA MECÂNICA,5.7,518
4,Concluinte,ENGENHARIA DE COMPUTAÇÃO,5.6,481
0,Concluinte,ADMINISTRAÇÃO,4.6,477
6,Concluinte,ENGENHARIA DE MATERIAIS,5.6,400
1,Concluinte,ENGENHARIA AMBIENTAL E SANITÁRIA,5.8,268


In [18]:
# Filtrar para obter apenas a maior contagem de alunos para ingressante e para concluinte
df_maior_contagem = df_grupo.loc[df_grupo.groupby('Ingressante/Concluinte')['Contagem de Alunos'].idxmax()]

# Função de String: converter a coluna 'Curso' com a inicial em maiúsculo
df_maior_contagem['Curso'] = df_maior_contagem['Curso'].str.capitalize()

# Exibir o resultado
display(df_maior_contagem)

,Ingressante/Concluinte,Curso,Tempo Médio de Conclusão (anos),Contagem de Alunos
12,Concluinte,Engenharia industrial mecânica,5.7,1322
24,Ingressante,Engenharia de computação,NaN,182


In [20]:
curso_max_ingres = df_maior_contagem['Curso'].iloc[1]
cont_max_ingres = df_maior_contagem['Contagem de Alunos'].iloc[1]
curso_max_conc = df_maior_contagem['Curso'].iloc[0]
cont_max_conc = df_maior_contagem['Contagem de Alunos'].iloc[0]
tempo_conclusao = df_maior_contagem['Tempo Médio de Conclusão (anos)'].iloc[0]

# Resultado final:
print(f'O curso com a maior número de ingressantes é {curso_max_ingres} com {cont_max_ingres} alunos.') 
print(f'O curso com o maior numero de concluintes é {curso_max_conc} com {cont_max_conc} alunos e um tempo médio de conclusão de {tempo_conclusao} anos.')

O curso com a maior número de ingressantes é Engenharia de computação com 182 alunos.
O curso com o maior numero de concluintes é Engenharia industrial mecânica com 1322 alunos e um tempo médio de conclusão de 5.7 anos.


# Resultado final: 
A princípio, pensei em manter apenas a resposta dos alunos concluíntes por ter um maior número de dados. 
Porém, apesar de termos duas informações, em minha análise, acredito que fez sentido apresentar uma resposta única para cada tipo de aluno, mantendo assim, sua exclusividade e relevância.